In [1]:
import matplotlib
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_location = '~/Desktop/cleanedFraudData.csv'

In [2]:
pd.set_option('display.max_columns', None)
df = pd.read_csv(data_location)
df.head()

,category,amt,is_fraud,transaction_area,merchant_area
0,8,4.97,0,3,4
1,4,107.23,0,2,2
2,0,220.11,0,7,6
3,2,45.00,0,7,6
4,9,41.96,0,0,5


In [3]:
from sklearn.model_selection import train_test_split

X, y = df.drop('is_fraud', axis=1), df[['is_fraud']]

In [4]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [5]:
import xgboost as xgb

dtrain_reg = xgb.DMatrix(X_train, y_train)
dtest_reg = xgb.DMatrix(X_test, y_test)

C:\ProgramData\anaconda3\envs\capstone\Lib\site-packages\xgboost\data.py:335: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(dtype):
C:\ProgramData\anaconda3\envs\capstone\Lib\site-packages\xgboost\data.py:338: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  is_categorical_dtype(dtype) or is_pa_ext_categorical_dtype(dtype)
C:\ProgramData\anaconda3\envs\capstone\Lib\site-packages\xgboost\data.py:384: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  if is_categorical_dtype(dtype):
C:\ProgramData\anaconda3\envs\capstone\Lib\site-packages\xgboost\data.py:359: FutureWarning: is_categorical_dtype is deprecated and will be removed in a future version. Use isinstance(dtype, CategoricalDtype) instead
  return is_int

In [10]:
params = {"objective": "binary:logistic"}

evals = [(dtrain_reg, "train"), (dtest_reg, "validation")]

model = xgb.train(
   params=params,
   dtrain=dtrain_reg,
   num_boost_round=1000,
   evals=evals,
   verbose_eval=25,
   early_stopping_rounds=50
)

C:\ProgramData\anaconda3\envs\capstone\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:54:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\common\error_msg.cc:27: The tree method `gpu_hist` is deprecated since 2.0.0. To use GPU training, set the `device` parameter to CUDA instead.

    E.g. tree_method = "hist", device = "cuda"

  warnings.warn(smsg, UserWarning)
C:\ProgramData\anaconda3\envs\capstone\Lib\site-packages\xgboost\core.py:160: UserWarning: [18:54:17] WARNING: C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\context.cc:44: No visible GPU is found, setting device to CPU.
  warnings.warn(smsg, UserWarning)


XGBoostError: [18:54:17] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\tree\updater_gpu_hist.cu:781: Exception in gpu_hist: [18:54:17] C:\buildkite-agent\builds\buildkite-windows-cpu-autoscaling-group-i-0cec3277c4d9d0165-1\xgboost\xgboost-ci-windows\src\tree\updater_gpu_hist.cu:787: Check failed: ctx_->gpu_id >= 0 (-1 vs. 0) : Must have at least one device


In [7]:
results = xgb.cv(
   params, dtrain_reg,
   num_boost_round=75, #75
   nfold=2,
   metrics=["aucpr"]
)

results['test-aucpr-mean'].max()

0.7132067325589487